# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
</div>
****

In [53]:
import pandas as pd
import numpy as np
from scipy import stats

In [54]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [55]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


### Choose the test and state the hypotheses  
We want to find out whether the average call back rate for black-sounding names is statistically different from that of white-sounding names. Since the sample size is quite large, the CLT applies and we can assume a normal distribution and test our hypothesis using the z-statistic.   

We can formulate the following hypotheses:  
$H_0$: $\mu_b$ - $\mu_w$ = 0  
$H_A$: $\mu_b$ - $\mu_w$ < 0  

Our test statistic is the difference between the mean call back rates for black-sounding names and white-sounding names.

First, we split the data into 2 subsets for black-sounding names and white-sounding names

In [56]:
b = data[data.race =='b']
w = data[data.race =='w']

Then we calculate the z-statistic 

In [60]:
mean_sample = b.call.mean() - w.call.mean() #Sample mean of the sampling distribution
std_error_sample = np.sqrt(b.call.std()**2/len(b) + w.call.std()**2/len(w)) #Std error of the sampling distribution
z = mean_sample/std_error_sample #z-score
pval = stats.t.sf(abs(z), len(b)-1)  #p-value for one-tailed test

print ('z-statistic =', z , 'and p-value =', pval)

z-statistic = -4.11473832443 and p-value = 2.00252681786e-05


With a significance level of 5%, the critical z-score for a one-tailed test is 1.65. Since the z-statistic we got is higher than that (*abs*(-4.11) > 1.65, we can reject the null hypothesis in favor of the alternative, and conclude that the average call back rate for black-sounding names is statistically different from (and less than) that of white-sounding names.

### Calculate 95% confidence interval
Next, we can calculate the 95% confidence interval for the difference between callback rates for black-sounding and white-sounding names:

In [61]:
print ('95 percent confidence interval is:', (mean_sample - 1.96*std_error_sample, mean_sample + 1.96*std_error_sample))

95 percent confidence interval is: (-0.047291272417895609, -0.016774437292225546)


### Does the analysis mean that race/name is the most important factor in callback success?

We can't conclude that race/name is the most important factor in callback success though, because we haven't controlled for other variables that may not be correlated with callback. In order to find out which factors also affect callback rate and their magnitude, we need to run a regression with some other control variables such as education, years of experience, etc...